# Wine Quality

This lab is based in the AWS-course for machine learning foundations, module 3, section 1, first question:

Question: Based on the composition of the wine, can you predict the quality and therefore the price?

Why:
* View statistics
* Deal with outliers
* Scale numeric data

Citation
Source: [UCI Wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) _-> already downloaded!_

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science. 

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.


Step 1: load both files (you can ignore the names-file) in one dataframe. __One-hot encode the color of the wine!__ Check out the shape and NA's of the dataset.

In [ ]:
import pandas as pd

wine = pd.read_csv('files/wine+quality/winequality-red.csv', sep=';')
wine["red"] = 1
wine["white"] = 0
white_wine = pd.read_csv('files/wine+quality/winequality-white.csv', sep=';')
white_wine["red"] = 0
white_wine["white"] = 1

wine = pd.concat([wine, white_wine], axis=0, ignore_index=True)

print(wine.shape)
print(wine.info())

Any outliers? Start with describe...

In [ ]:
#DELETE

wine.describe()

A lot of numbers. For example this column:

![](files/2023-06-13-10-47-12.png)

Is fine. The max value of 4.010000 is not far from the mean and three times the standard deviation of 3.700862. And this is good because:

![](files/2023-06-13-10-49-36.png)

It means this is quite near the edges of the bell curve. (Image from [wikipedia](https://en.wikipedia.org/wiki/Standard_deviation).)

But looking at:

![](files/2023-06-13-10-50-57.png)

We see another problem. 289 is very far from 83.8. This indicates an outlier.

In [ ]:
mean_and_std = 3.218501 + 3 * 0.160787
print(mean_and_std)

mean_and_std = 30.525319 + 3* 17.749400
print(mean_and_std)


Let's do some box plots! Plot a box plot for every column (except for red and white) and don't share tha y-axis.

(See what we are doing here? We are condensing a lot of number into a few drawings. We get less information but our brain will be able to process it way easier.)

In [ ]:
#DELETE

import matplotlib.pyplot as plt

fig, ax = plt.subplots(ncols=4, nrows=3, sharey=False, figsize=(20, 20))

for i, (column) in enumerate(wine.columns[:-2]):
    # print(i, column, i//4, i%4)
    ax[i//4, i%4].boxplot(wine[column])
    ax[i//4, i%4].set_title(column)

Looks like we were on the right track with pH and free sulfur dioxide. pH has a big box and few outliers, free sulfur dioxide has one major outlier. We could have done better though, looking at the box plots alcohol has next to no outliers, and chlorides a _lot_ of them.

Also note how there are little to no outliers on the low end, so we can safely ignore the minimal-values.

Next up it would be nice to see if the outlier for volatile acidity, citric acid,residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide and density (they all have one very distinguished outlier) are the same wine. This could indicate a badly measured wine. If not we could live with dropping all of those outliers, as we have 6497 records.

Let's show the records first and go from there. Show the entire row, but make sure you know which is the column that is responsible.

In [ ]:
#DELETE

print_df = pd.DataFrame()

for i, (column) in enumerate(wine.columns[:-2]):
    # temp_df.append(wine[wine[column]==wine[column].max()])
    temp_df = wine[wine[column]==wine[column].max()].copy()
    temp_df["max_col"] = column
    print_df = pd.concat([print_df, temp_df], axis=0, ignore_index=True)

print_df.sort_values(by="density")

If you sort the above result by a column with long values, like density, you'll note that some rows are in there twice. For following values, the same wines have the maximum value:

* free and total sulfur dioxide
* alcohol and fixed acidity
* density and residual sugar

This can also be shown a multivariate scatterplot. Draw one for these three combinations.

In [ ]:
#DELETE

import matplotlib.pyplot as plt

fig, ax = plt.subplots(ncols=3, figsize=(18, 5))

ax[0].scatter(wine["free sulfur dioxide"], wine["total sulfur dioxide"], s=0.3)
ax[0].set_title("free and total sulfur dioxide")
ax[1].scatter(wine["alcohol"], wine["fixed acidity"], s=0.3)
ax[1].set_title("alcohol and fixed acidity")
ax[2].scatter(wine["density"], wine["residual sugar"], s=0.3)
ax[2].set_title("density and residual sugar")


In each graph we see the dot in the upper right that we detected. Correlation is pretty good for the first and last graph, but not that great for the middle graph. We'll dive into correlation later on.

This correlation means we won't be deleting 16 rows but slightly less. Especially if we don't remove the lines with the best quality: there are 4 of them and as that is the trait we want to maximize, we really want to keep these rows.

Do you think we should proceed on this course? Delete the 10-ish rows, look at the boxplots, maybe delete some more? Were will that lead us?

Turn on some [thinking music](https://www.youtube.com/watch?v=oPVte6aMprI) and reminiscence on this.

Don't start deleting lines. Use statiscal analysis. The [Z-score](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html) of a value says if it's outside of the 3-quartile range, so any data out of this range will surely be an outlier. Let's check how much data we'd lose if applying this to all columns except for quality (and red and white, but they don't matter, as you can surely explain).

The results will be to big to show here. Export them to a CSV to investigate.

In [ ]:
#DELETE

from scipy import stats

print_df = pd.DataFrame()

for i, (column) in enumerate(wine.columns[:-3]):
    temp_df = wine[stats.zscore(wine[column]) > 3].copy()
    temp_df["max_col"] = column
    print_df = pd.concat([print_df, temp_df], axis=0, ignore_index=True)

print_df.to_csv("outliers.csv")
print_df.sort_values(by="max_col")

# wine[(np.abs(stats.zscore(wine)) > 3).any(axis=1)]

Group the rows you're about to delete by the column based on which they'll be deleted. How many are there per type?

In [ ]:
#DELETE

print_df.groupby(by="max_col").count()

You'll be deleting a lot because of chlorides and fixed acidity, so you could reconsider these. Alcohol, density and total sulfur dioxide are no-brainers.

But hey, lets delete them all and look at the box plots again.

In [ ]:
#DELETE

from scipy import stats

print_df = pd.DataFrame()

for i, (column) in enumerate(wine.columns[:-3]):
    wine = wine[stats.zscore(wine[column]) < 3]

wine.info()

In [ ]:
#DELETE

import matplotlib.pyplot as plt

fig, ax = plt.subplots(ncols=4, nrows=3, sharey=False, figsize=(20, 20))

for i, (column) in enumerate(wine.columns[:-2]):
    # print(i, column, i//4, i%4)
    ax[i//4, i%4].boxplot(wine[column])
    ax[i//4, i%4].set_title(column)

That's what outlier-free data is supposed to look like!

Next up is scaling. Alcohol is between 14 and 8, which is higher than sulphates which are between 0.2 and 0.9. For an AI-model this is confusing, as it seems that alcohol is 10 times more important than sulphates. This process is called normalisation, and it means we recalculate all values to a scale of -1 to +1, or of 0 to 1. You can read all about it [here](https://datagy.io/pandas-normalize-column/).

But we'll leave that for another day.

What we will be doing though is a correlation heatmap. Make sure you can read the numbers (i.e. that the image is big enough).

(Look up how  to on the [internet](https://datagy.io/python-correlation-matrix/)!)

In [ ]:
#DELETE

import seaborn as sns
from matplotlib.pyplot import figure

figure(figsize=(12, 12), dpi=80)

matrix = wine.corr(
    method = 'pearson',  # The method of correlation
    min_periods = 1      # Min number of observations required
).round(decimals=2)


sns.heatmap(matrix, annot=True)
plt.show()

# print(matrix)